In [ ]:
%pip install pandas pyarrow
%pip install tqdm
%pip install plotly
%pip install nbformat
%pip install ipywidgets
%pip install numba

In [ ]:
# Function to download data.

import requests
from tqdm import tqdm

def get_filename(ticker, interval, year):
    return f'data/{ticker}_{interval}_{year}.parquet'

def download(ticker, interval, year):
    filename = get_filename(ticker, interval, year)
    # Create directory if it doesn't exist
    os.makedirs(os.path.dirname(filename), exist_ok=True)
    if year:
      # URL for seconds interval
      url = f"https://barsv.com/api/download/file/{ticker}/{year}?format=parquet&interval={interval}"
    else:
      # URL for minute+ intervals
      url = f"https://barsv.com/api/download/file/{ticker}/{ticker}_{interval}?interval={interval}&format=parquet"
    response = requests.get(url, stream=True)

    # Get the total file size from the content-length header
    total_size = int(response.headers.get('content-length', 0))

    with open(filename, 'wb') as f, tqdm(
        desc=filename,
        total=total_size,
        unit='iB',
        unit_scale=True,
        unit_divisor=1024,
    ) as bar:
        for data in response.iter_content(chunk_size=1024):
            size = f.write(data)
            bar.update(size)

In [ ]:
ticker = 'AAPL'
interval = '5s'
year = '2024'

In [ ]:
import os

filename = get_filename(ticker, interval, year)
# If file doesn't exist, then download it.
if not os.path.exists(filename):
	download(ticker, interval, year)

In [ ]:
# Plot last points.
import pandas as pd
import plotly.express as px

df = pd.read_parquet(filename)

# fig = px.line(df[-1000:], y='open', title=f'{ticker} Open Prices')
# fig.show()

In [ ]:
total_bars = df.shape[0]
print(f'Total bars: {total_bars} ({total_bars:,})')

In [ ]:
# Set trailing stop-loss levels
import numpy as np

max_sl = 1
sl_steps = 10
sl_step = max_sl / sl_steps
stop_loss_percents = np.concatenate((np.arange(-max_sl, 0, sl_step), np.arange(sl_step, max_sl + sl_step, sl_step)))
stop_loss_levels = stop_loss_percents / 100

print("Stop-loss levels (in percent):")
print(stop_loss_percents)
print("\nStop-loss levels (as fractions):")
print(stop_loss_levels)

In [ ]:
import numba

@numba.jit(nopython=True)
def calculate_pl_numba(df_open, df_low, df_high, df_close, stop_loss_levels):
    """
    Numba-optimized function for calculating P/L with trailing stop-loss.
    """
    num_entries = len(df_open)
    num_sl_levels = len(stop_loss_levels)
    # Numba does not work well with list-of-dicts, so we collect results in a NumPy array
    results_np = np.empty(( (num_entries - 1) * num_sl_levels, 4)) # index, sl_level, profit, entry_price
    result_idx = 0

    # Iterate over each entry in the experimental dataframe (except the last one)
    for i in range(num_entries - 1):
        entry_price = df_open[i]

        # Iterate over each stop-loss level
        for sl_idx in range(num_sl_levels):
            sl_level = stop_loss_levels[sl_idx]
            if sl_level == 0:
                continue

            stop_price = entry_price * (1 + sl_level)
            exit_price = 0.0
            trade_closed = False

            # Search for the moment when stop-loss is triggered in the future
            for j in range(i + 1, num_entries):
                future_low = df_low[j]
                future_high = df_high[j]

                # For long positions (sl_level < 0)
                if sl_level < 0:
                    if future_low <= stop_price:
                        exit_price = stop_price
                        trade_closed = True
                        break
                    else:
                        new_stop_price = future_high * (1 + sl_level)
                        if new_stop_price > stop_price:
                            stop_price = new_stop_price
                # For short positions (sl_level > 0)
                elif sl_level > 0:
                    if future_high >= stop_price:
                        exit_price = stop_price
                        trade_closed = True
                        break
                    else:
                        new_stop_price = future_low * (1 + sl_level)
                        if new_stop_price < stop_price:
                            stop_price = new_stop_price

            if not trade_closed:
                exit_price = df_close[-1]

            profit = 0.0
            if sl_level < 0:
                profit = 100 * (exit_price - entry_price) / entry_price
            elif sl_level > 0:
                profit = 100 * (entry_price - exit_price) / entry_price

            results_np[result_idx, 0] = i
            results_np[result_idx, 1] = sl_level
            results_np[result_idx, 2] = profit
            results_np[result_idx, 3] = entry_price

            result_idx += 1

    return results_np[:result_idx] # Return only the filled part of the array

def calculate_pl(df_exp, sl_lvels):
    df_open_np = df_exp['open'].to_numpy()
    df_low_np = df_exp['low'].to_numpy() 
    df_high_np = df_exp['high'].to_numpy()
    df_close_np = df_exp['close'].to_numpy()

    results_2d = calculate_pl_numba(df_open_np, df_low_np, df_high_np, df_close_np, sl_lvels)
    
    # Более читаемая распаковка
    i_np = results_2d[:, 0].astype(int)
    sl_level_np = results_2d[:, 1] 
    profit_np = results_2d[:, 2]
    entry_price_np = results_2d[:, 3]
    
    results_df = pd.DataFrame({
        'index': i_np,
        'sl_level': sl_level_np,
        'profit': profit_np, 
        'entry_price': entry_price_np
    })
    
    return results_df


In [ ]:
# Step 4: Visualization of results
import plotly.graph_objects as go

def plot_sl_profits(df_exp, results_df):
    fig = go.Figure()

    # Add stop-loss points
    # For better performance, we can add all points in one call
    # Filter out zero SL, as they do not carry information
    plot_df = results_df.copy()#[results_df['sl_level'] != 0].copy()
    plot_df['sl_price'] = plot_df['entry_price'] * (1 + plot_df['sl_level'])

    min_profit = plot_df['profit'].min()
    max_profit = plot_df['profit'].max()

    # Calculate normalized position for zero (between 0 and 1)
    zero_position = (0 - min_profit) / (max_profit - min_profit)

    custom_colorscale = [
        [0.0, 'red'],           # Minimum profit = red
        [zero_position, 'white'], # Zero profit = white
        [1.0, 'blue']           # Maximum profit = blue
    ]

    fig.add_trace(go.Scatter(
        x=plot_df['index'],
        y=plot_df['sl_price'],
        mode='markers',
        marker=dict(
            size=3,
            color=plot_df['profit'],
            colorscale=custom_colorscale,
            showscale=True,
            colorbar=dict(title='Profit (%)'),
            cmin=min_profit,  # Set the actual data range
            cmax=max_profit
        )
    ))

    # Add OHLC prices on top
    fig.add_trace(go.Scatter(
        # x=df_exp['timestamp'],
        y=df_exp['open'],
        mode='lines',
        line=dict(color='black', width=1)
    ))
    fig.add_trace(go.Scatter(
        y=df_exp['high'],
        mode='lines',
        line=dict(color='black', width=1)
    ))
    fig.add_trace(go.Scatter(
        y=df_exp['low'],
        mode='lines',
        line=dict(color='black', width=1)
    ))

    fig.update_layout(
        title="Trailing SL profits",
        showlegend=False
    )

    fig.show()

In [ ]:
# calculate for all points. slice results later.
shift = 0#-140 * 1000
window = len(df) #100 * 1000
df_exp = df[shift:shift + window].copy()
results_df = calculate_pl(df_exp, stop_loss_levels)

In [ ]:
results_df

In [ ]:
def slice_results_by_time(results_df, start_idx=None, end_idx=None, num_points=None):
    """
    Slice results by time indices (not by DataFrame rows).
    
    Parameters:
    - start_idx, end_idx: time indices (not row indices)
    - num_points: alternative to end_idx, take N time points from start_idx
    """
    if num_points is not None:
        end_idx = start_idx + num_points if start_idx is not None else num_points
    
    mask = True
    if start_idx is not None:
        mask &= (results_df['index'] >= start_idx)
    if end_idx is not None:
        mask &= (results_df['index'] < end_idx)
    
    sliced_df = results_df[mask].copy()
    
    # Normalize indices to start from 0
    if start_idx is not None and len(sliced_df) > 0:
        sliced_df['index'] = sliced_df['index'] - start_idx
    
    return sliced_df

In [ ]:
plot_shift = 75*1000
plot_window = 10*1000
plot_df = slice_results_by_time(results_df, start_idx=plot_shift, num_points=plot_window)
plot_sl_profits(df_exp[plot_shift:plot_shift + plot_window], plot_df)

In [ ]:
plot_shift = 85*1000
plot_window = 10*1000
plot_df = slice_results_by_time(results_df, start_idx=plot_shift, num_points=plot_window)
plot_sl_profits(df_exp[plot_shift:plot_shift + plot_window], plot_df)

In [ ]:
import pandas as pd
import plotly.graph_objects as go

# Custom plot function showing specified percentiles and the mean for each category
def box(df, x, y, title=None, percentiles=[0, 10, 25, 50, 75, 90, 100]):
    """
    Custom box-like plot function showing specified percentiles and the average for each category.
    
    Parameters:
    df         : pandas.DataFrame
    x          : str, column name for categories
    y          : str, column name for values
    title      : str, plot title
    percentiles: list of ints, percentiles to display (e.g., [0,10,25,50,75,90,100])
    """
    # Ensure percentiles are sorted and converted to fractions
    percentiles_sorted = sorted(percentiles)
    quantile_fracs = [p / 100 for p in percentiles_sorted]

    # Compute the specified percentile values for each group
    grouped_q = df.groupby(x)[y].quantile(quantile_fracs).unstack()
    # Compute the mean value for each group
    grouped_mean = df.groupby(x)[y].mean()

    # Initialize figure
    fig = go.Figure()

    # Add a trace for each percentile
    for p, frac in zip(percentiles_sorted, quantile_fracs):
        fig.add_trace(go.Scatter(
            x=grouped_q.index,
            y=grouped_q[frac],
            mode='lines+markers',
            name=f'{p}th percentile',
            line=dict(dash='solid' if p == 50 else 'dash'),
            marker=dict(size=6)
        ))

    # Add mean as a bold black line
    fig.add_trace(go.Scatter(
        x=grouped_mean.index,
        y=grouped_mean.values,
        mode='lines',
        name='Mean',
        line=dict(color='black', width=4),
        hovertemplate='Mean: %{y:.2f}<extra></extra>'
    ))

    # Update layout
    fig.update_layout(
        title=title,
        xaxis_title=x,
        yaxis_title=y,
        legend_title='Metrics',
        template='plotly_white'
    )

    return fig

# Example usage:
# fig = box(results_df, x='sl_level', y='profit',
#           title='Profit Distribution by Stop-Loss Level',
#           percentiles=[0, 10, 25, 50, 75, 90, 100])
# fig.show()


In [ ]:
# Example usage
fig = box(results_df, x='sl_level', y='profit',
    title='Profit Distribution by Stop-Loss Level',
    percentiles=[0, 25, 50, 75, 90, 100])
fig.show()
